In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

In [2]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

In [3]:
train_x.shape

(60000, 28, 28)

In [4]:
train_x = train_x.reshape([-1, 28*28])
test_x = test_x.reshape([-1, 28*28])

In [5]:
train_y_eye = np.eye(10)[train_y]
test_y_eye = np.eye(10)[test_y]

In [6]:
train_x = train_x/255.
test_x = test_x/255.

In [7]:
learning_rate = 0.02
epoch = 500
n_batch = 20
batch_size = int(len(train_x)/n_batch)
display_step = 20
activation = tf.nn.relu


In [8]:
with tf.device('device:GPU:0'):
    
    tf.reset_default_graph()
    
    X = tf.placeholder(dtype = tf.float32, shape = [None, 28*28])
    Y = tf.placeholder(dtype = tf.float32, shape = [None, 10])
    
    keep_prob = tf.placeholder(dtype = tf.float32, shape = [])
    
    hidden1 = tf.layers.dense(X, 1024, activation=activation)
    dropout1 = tf.nn.dropout(hidden1, keep_prob=keep_prob)
    
    hidden2 = tf.layers.dense(dropout1, 2048, activation=activation)
    dropout2 = tf.nn.dropout(hidden2, keep_prob=keep_prob)
    
    logit = tf.layers.dense(dropout2, 10)
    pred = tf.nn.softmax(logit)
    
    loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(Y, logit))
    #
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1)),tf.float32))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [9]:
losses = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(epoch):
        for i in range(n_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_y = train_y_eye[i*batch_size:(i+1)*batch_size]
            
            _, l = sess.run([optimizer, loss], feed_dict = {X:batch_x, Y:batch_y, keep_prob:0.9})
        losses.append(l)
        if (step+1) % display_step == 0:
            print(f'Step {step+1} loss currently at {l}')
    
    acc = sess.run(accuracy, feed_dict = {X:test_x, Y:test_y_eye, keep_prob:1.})
    print(f'Accuracy on the test set: {acc}')

Step 20 loss currently at 0.07736432552337646
Step 40 loss currently at 0.05624712258577347
Step 60 loss currently at 0.044727690517902374
Step 80 loss currently at 0.03283767029643059
Step 100 loss currently at 0.043731629848480225
Step 120 loss currently at 0.036375030875205994
Step 140 loss currently at 0.04134982079267502
Step 160 loss currently at 0.046827297657728195
Step 180 loss currently at 0.04055628925561905
Step 200 loss currently at 0.027607444673776627
Step 220 loss currently at 0.03874732553958893
Step 240 loss currently at 0.04285433515906334
Step 260 loss currently at 0.04339078813791275
Step 280 loss currently at 0.037821076810359955
Step 300 loss currently at 0.04249660670757294
Step 320 loss currently at 0.04288775101304054
Step 340 loss currently at 0.042931366711854935
Step 360 loss currently at 0.04199666529893875
Step 380 loss currently at 0.032364651560783386
Step 400 loss currently at 0.03776206821203232
Step 420 loss currently at 0.028302719816565514
Step 440